# Dash app basics

## What is dash?
[Dash](https://dash.plot.ly/)
is a python framework for building simple web apps.

It combines:

* [Plotly](https://plot.ly/) for interactive graphs
* [Flask](http://flask.pocoo.org/) for the web framework
* [React](https://reactjs.org/) for the javascript user interface

We'll take a bare bones look at an example Python Dash app today.

First, set up your environment:

`conda env create -f environment.yml`

See [creating conda environments from an environment.rml file](https://conda.io/docs/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file). There's an example environment.yml file available [here](https://github.com/pyladiesmelbourne/dash-app-basics/blob/master/environment.yml)

## The Gapminder dataset
The dataset used here is based on free material from
[GAPMINDER.ORG](https://www.gapminder.org/data/), CC-BY LICENSE.

In [6]:
import pandas as pd
url = "https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv"
df = pd.read_csv(url)
df = df.rename(index=str, columns={"pop": "population",
                                   "lifeExp": "life_expectancy",
                                   "gdpPercap": "GDP_per_capita"})
df.sample(5)

,country,year,population,continent,life_expectancy,GDP_per_capita
554,Gambia,1962,374020.0,Africa,33.896,599.650276
55,Argentina,1987,31620918.0,Americas,70.774,9139.671389
255,Central African Republic,1967,1733638.0,Africa,41.478,1136.056615
715,Indonesia,1987,169276000.0,Asia,60.137,1748.356961
1422,Spain,1982,37983310.0,Europe,76.300,13926.169970


## Graphing with Plotly
[https://plot.ly/python/getting-started/](https://plot.ly/python/getting-started/)

Plotly graphs have a ton of built in interactive features you can use.
Here's the 'Hello World' of a basic line chart. 

In [8]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [9]:
plotly.offline.iplot({
    "data": [go.Scatter(
        x=[1, 2, 3, 4],
        y=[4, 3, 2, 1]
    )],
    "layout": go.Layout(
        title="hello world"
    )
})

Now let's plot a time series of GDP per capita for a variety of user-selected countries. We'll start with Australia and New Zealand.

In [10]:
# user input
country_values = ["Australia", "New Zealand"]

In [16]:
dff = df.loc[df['country'].isin(country_values)]

plotly.offline.iplot({
    "data": [go.Scatter(
            x=dff[dff['country'] == country]['year'],
            y=dff[dff['country'] == country]['GDP_per_capita'],
            text="Continent: " +
                  f"{dff[dff['country'] == country]['continent'].unique()[0]}",
            mode='lines+markers',
            name=country,
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        ) for country in dff.country.unique()],
    "layout": go.Layout(
            xaxis={'title': 'Year'},
            yaxis={'title': 'GDP Per Capita'},
            margin={'l': 60, 'b': 50, 't': 10, 'r': 0},
            hovermode='closest'
    )
})

## Putting it into a dash app

First we'll talk about the structure of a dash app.

[Getting started with dash](https://dash.plot.ly/getting-started), there are three main concepts:

* [Layout](https://dash.plot.ly/getting-started)
* [Callbacks](https://dash.plot.ly/getting-started-part-2)
* [State](https://dash.plot.ly/state)

**Layout** define how the app will look, and how the elements are arranged on the screen. 

**Callbacks** control how user interaction can modify elements (like graphs updating when you move a slider).

**State** is analogous to memory, and is useful if the order in which the user performs some actions is important. This is beyond the scope of this talk, but there's more information [here](https://dash.plot.ly/state) if you want to dive in.

We'll focus on both layout and callbacks today, so we don't run overtime.

In [ ]:
# STRUCTURE OF A DASH APP - app.py

# imports
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go

# load data
url = "https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv"
df = pd.read_csv(url)

# initialize dash app
app = dash.Dash()
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

# layout
# ...

# callbacks
# ...

if __name__ == '__main__':
    app.run_server(debug=True)

## Layout
If you're familiar with html, you'll feel pretty comfy with how to control the layout of a dash app. You can arrange html components (divisions, headings, paragraphs), as well as dash core components (graphs, tables, dropdown boxes, sliders, etc.).

Anything you'll need to refer to later must have an id tag. This is how you'll set up the inputs and outputs for callbacks.

In [ ]:
import dash_html_components as html
import dash_core_components as dcc

from textwrap import dedent
...

app.layout = html.Div([
    html.H1('A heading!',
        id='heading-1'
    ),

    html.H2('Subheading here'
        id='subheading-1'
    ),

    dcc.Markdown(dedent("""
    ## Markdown is supported by dash, too
    If you prefer writing markdown over writing text in html,
    the dash core components support that too.
    
    > quotes
    
    ```
    code blocks
    ```
    """)),

    dcc.Graph(id='graph-object'),

    dcc.Dropdown(
        id='dropdown-1',
        options=[{'Option 1': 1, 'Option 2': 2, 'Option 3': 3}],
        multi=True, # option to allow multiple selections at once
        value=[1]   # option to set the initial value, instead of a blank
    )
])

Let's look at the graph object more closely...

In [ ]:
dcc.Graph(id='graph-object')  # right now, there's nothing here

We can add the plotly graph for some content. But, this is still static!

In [ ]:
dcc.Graph(id='graph-object',
    "data": [go.Scatter(
            x=dff[dff['country'] == country]['year'],
            y=dff[dff['country'] == country]['GDP_per_capita'],
            text="Continent: " +
                  f"{dff[dff['country'] == country]['continent'].unique()[0]}",
            mode='lines+markers',
            name=country,
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        ) for country in dff.country.unique()],
    "layout": go.Layout(
            xaxis={'title': 'Year'},
            yaxis={'title': 'GDP Per Capita'},
            margin={'l': 60, 'b': 50, 't': 10, 'r': 0},
            hovermode='closest'
    )
})

## Callbacks
Since we want user interactivity for our graph, we need to implement a callback. This means shifting the plotly graph *out* of the layout, and into the callback section.

A refresher:

In [ ]:
# STRUCTURE OF A DASH APP - app.py
# imports
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go

# load data
url = "https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv"
df = pd.read_csv(url)

# initialize dash app
app = dash.Dash()
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

# layout
app.layout = html.Div([
    html.H1('Dash App Basics', id='heading-1'),
    dcc.Markdown("")
    dcc.Dropdown(
        id='dropdown-1',
        options=[{'Option 1': 1, 'Option 2': 2, 'Option 3': 3}],
        multi=True, # option to allow multiple selections at once
        value=[1]   # option to set the initial value, instead of a blank
    )
    dcc.Graph(id='graph-object') 
    
# callbacks
# Take input drom dcc.Dropdown, and return a changed output to dcc.Graph

if __name__ == '__main__':
    app.run_server(debug=True)

To create a callback, we decorate the app function:

In [ ]:
@app.callback(
    dash.dependencies.Output('graph-object', 'figure'),
    [dash.dependencies.Input('dropdown-1', 'value')])
def update_graph(input_values):
    # create updated graph here 
    return updated_graph


There can be multiple inputs into any one callback function.

In [ ]:
@app.callback(
    dash.dependencies.Output('graph-object', 'figure'),
    [dash.dependencies.Input('dropdown-1', 'value')])
def update_graph(country_values):
    dff = df.loc[df['country'].isin(country_values)]
    return {
        'data': [go.Scatter(
            x=dff[dff['country'] == country]['year'],
            y=dff[dff['country'] == country]['GDP_per_capita'],
            text="Continent: " +
                  f"{dff[dff['country'] == country]['continent'].unique()[0]}",
            mode='lines+markers',
            name=country,
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        ) for country in dff.country.unique()],
        'layout': go.Layout(
            xaxis={'title': 'Year'},
            yaxis={'title': 'GDP Per Capita'},
            margin={'l': 60, 'b': 50, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }

# Running the app locally

To launch the dash app locally, run the python script. This will trigger the app.run_server command in main, since we had...
```
if __name__ == '__main__':
    app.run_server(debug=True) 
```

At the terminal, type:
`$ python app.py`

You should see something like:
```
* Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 234-967-415
```

Head to [http://127.0.0.1:8050/](http://127.0.0.1:8050/) to see your app.

Here are those pieces put together:

In [ ]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go
from textwrap import dedent

# Gapminder dataset GAPMINDER.ORG, CC-BY LICENSE
url = "https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv"
df = pd.read_csv(url)
df = df.rename(index=str, columns={"pop": "population",
                                   "lifeExp": "life_expectancy",
                                   "gdpPercap": "GDP_per_capita"})

# Dash app
app = dash.Dash()
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

app.layout = html.Div([
    html.H1('Dash App Basics',
    ),

    dcc.Markdown("Here's a bare bones look at an example Python Dash app."
    ),

    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': i, 'value': i} for i in df.country.unique()],
        multi=True,
        value=['Australia']
    ),

    dcc.Graph(id='timeseries-graph')

])

@app.callback(
    dash.dependencies.Output('timeseries-graph', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')])
def update_graph(country_values):
    dff = df.loc[df['country'].isin(country_values)]

    return {
        'data': [go.Scatter(
            x=dff[dff['country'] == country]['year'],
            y=dff[dff['country'] == country]['GDP_per_capita'],
            text="Continent: " +
                  f"{dff[dff['country'] == country]['continent'].unique()[0]}",
            mode='lines+markers',
            name=country,
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        ) for country in dff.country.unique()],
        'layout': go.Layout(
            title="GDP over time, by country",
            xaxis={'title': 'Year'},
            yaxis={'title': 'GDP Per Capita'},
            margin={'l': 60, 'b': 50, 't': 80, 'r': 0},
            hovermode='closest'
        )
    }

if __name__ == '__main__':
    app.run_server(debug=True)


![](images/DashAppScreenshot1.png)

## Adding more features
You can add more graphs, tables, and dropdowns or sliders for callbacks.

You can see the list of available components here:

* [Dash core components](https://dash.plot.ly/dash-core-components)
* [HTML components](https://dash.plot.ly/dash-html-components)

Let's add a table, and another graph to our Dash app. Now the app.py file looks like this:

In [ ]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go
from textwrap import dedent

# Gapminder dataset GAPMINDER.ORG, CC-BY LICENSE
url = "https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv"
df = pd.read_csv(url)
df = df.rename(index=str, columns={"pop": "population",
                                   "lifeExp": "life_expectancy",
                                   "gdpPercap": "GDP_per_capita"})

# Utility functions
def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )

# Dash app
app = dash.Dash()
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

app.layout = html.Div([
    html.H1('Dash App Basics',
    ),

    dcc.Markdown(dedent('''
    ## What is dash?
    [Dash](https://dash.plot.ly/)
    is a python framework for building simple web apps.

    It combines:

    * [Plotly](https://plot.ly/) for interactive graphs
    * [Flask](http://flask.pocoo.org/) for the web framework
    * [React](https://reactjs.org/) for the javascript user interface

    Here's a bare bones look at an example Python Dash app.
    ''')
    ),

    html.H2("The Gapminder dataset"),
    dcc.Markdown(dedent('''
    The dataset used here is based on free material from
    [GAPMINDER.ORG](https://www.gapminder.org/data/), CC-BY LICENSE.

    Let's take a look at the dataset...
    ''')
    ),
    dcc.Markdown(dedent('''
    ```
    import pandas as pd
    url = "https://raw.githubusercontent.com/plotly/datasets/master/" +
          "gapminderDataFiveYear.csv"
    df = pd.read_csv(url)
    df = df.rename(index=str, columns={"pop": "population",
                                       "lifeExp": "life_expectancy",
                                       "gdpPercap": "GDP_per_capita"})
    df.sample(5)
    ```
    ''')
    ),
    generate_table(df.sample(5)),

    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': i, 'value': i} for i in df.country.unique()],
        multi=True,
        value=['Australia']
    ),
    dcc.Graph(id='timeseries-graph'),

    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        id='year-slider',
        min=df['year'].min(),
        max=df['year'].max(),
        value=df['year'].min(),
        step=None,
        marks={str(year): str(year) for year in df['year'].unique()}
    )

])

@app.callback(
    dash.dependencies.Output('timeseries-graph', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')])
def update_graph(country_values):
    dff = df.loc[df['country'].isin(country_values)]

    return {
        'data': [go.Scatter(
            x=dff[dff['country'] == country]['year'],
            y=dff[dff['country'] == country]['GDP_per_capita'],
            text="Continent: " +
                  f"{dff[dff['country'] == country]['continent'].unique()[0]}",
            mode='lines+markers',
            name=country,
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        ) for country in dff.country.unique()],
        'layout': go.Layout(
            title="GDP over time, by country",
            xaxis={'title': 'Year'},
            yaxis={'title': 'GDP Per Capita'},
            margin={'l': 60, 'b': 50, 't': 80, 'r': 0},
            hovermode='closest'
        )
    }

@app.callback(
    dash.dependencies.Output('graph-with-slider', 'figure'),
    [dash.dependencies.Input('year-slider', 'value')])
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]
    traces = []
    for i in filtered_df.continent.unique():
        df_by_continent = filtered_df[filtered_df['continent'] == i]
        traces.append(go.Scatter(
            x=df_by_continent['GDP_per_capita'],
            y=df_by_continent['life_expectancy'],
            text=df_by_continent['country'],
            mode='markers',
            opacity=0.7,
            marker={
                'size': 15,
                'line': {'width': 0.5, 'color': 'white'}
            },
            name=i
        ))

    return {
        'data': traces,
        'layout': go.Layout(
            title="Correlation between GDP and life expectancy",
            xaxis={'type': 'log', 'title': 'GDP Per Capita'},
            yaxis={'title': 'Life Expectancy', 'range': [20, 90]},
            margin={'l': 40, 'b': 40, 't': 150, 'r': 10},
            legend={'x': 0, 'y': 1},
            hovermode='closest'
        )
    }

if __name__ == '__main__':
    app.run_server(debug=True)


![](images/DashAppScreenshot2.png)

## Sharing your dash app publically

Later, you'll want to share your app with the world!

There are a number of ways you can do that, one of which is to sign up to a free plan with [Heroku](https://www.heroku.com/).

Check out this template to get yourself up and running: 
https://github.com/plotly/dash-heroku-template


## Dash apps in the wild

Here's some inspiration for you: [7 New Dash Apps Made by the Dash Community](https://medium.com/@plotlygraphs/7-new-dash-apps-made-by-the-dash-community-196998112ce3)

### Traffidc Accidents in the UK

This dash app is hosted with Heroku, and shows patterns in the statistics of UK traffic accidents.
https://traffic-accidents-uk.herokuapp.com/

![](images/UK-traffic-accidents.png)

### Wind Speed Streaming

This dash app polls a SQL server at regular intervals to show you real-time streaming data of wind speed measurements. The whole thing is built with only a couple of hundred lines of Python.

https://dash-live-wind-data.plot.ly/

[Source code here](https://github.com/plotly/dash-wind-streaming)

In [4]:
from IPython.core.display import HTML
HTML('<img src="images/dash-wind-streaming.gif">')

## Now go forth, and have fun with dash!

Here are those useful resource links again:

* [Dash user guide](https://dash.plot.ly/)
* [Some datasets to play with](https://github.com/plotly/dash-app-datasets)
* [Graphing with Plotly](https://plot.ly/python/)
* [Heroku dash app template](https://github.com/plotly/dash-heroku-template)

Remember to come back and share with us what you build!